In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [8]:
from scipy.optimize import Bounds, minimize
from scipy.optimize import NonlinearConstraint

### Define objective function

In [4]:
def f(x):
    # TODO
    #func = -x[0] - x[1]
    return -x[0] - x[1]

### Define constraint functions and derivatives

#### 1. calculation process of cons_f(x):

transform the constraints to non-negative inequality constraints:
($x_1 \ge 0$ and $x_2 \ge 0$ are set in **bounds**)

$$\begin{bmatrix}
2x_1^4 - 8x_1^3 + 8x_1^2 - x_2 + 2\\
4x_1^4 - 32x_1^3 + 88x_1^2 - 96x_1 - x_2 + 36\\
-x_1 + 3\\
-x_2 + 4
\end{bmatrix}
\ge
\begin{bmatrix}
0\\
0\\
0\\
0
\end{bmatrix}$$

#### 2. calculation process of cons_Jacobian:

$$
C=
\begin{bmatrix}
c_1\\
c_2\\
c_3\\
c_4
\end{bmatrix}
=
\begin{bmatrix}
2x_1^4 - 8x_1^3 + 8x_1^2 - x_2 + 2\\
4x_1^4 - 32x_1^3 + 88x_1^2 - 96x_1 - x_2 + 36\\
-x_1 + 3\\
-x_2 + 4
\end{bmatrix}
$$

$$
\nabla C=
\begin{bmatrix}
\nabla c_1\\
\nabla c_2\\
\nabla c_3\\
\nabla c_4
\end{bmatrix}
=
\begin{bmatrix}
\frac{\partial c_1}{\partial x_1} & \frac{\partial c_1}{\partial x_2}\\
\frac{\partial c_2}{\partial x_1} & \frac{\partial c_2}{\partial x_2}\\
\frac{\partial c_3}{\partial x_1} & \frac{\partial c_3}{\partial x_2}\\
\frac{\partial c_4}{\partial x_1} & \frac{\partial c_4}{\partial x_2}
\end{bmatrix}
=
\begin{bmatrix}
8x_1^3 - 24x_1^2 + 16*x_1 & -1\\
16x_1^3 - 96x_1^2 + 176*x_1 - 96 & -1\\
-1 & 0\\
0 & -1
\end{bmatrix}
$$

#### 3. calculation process of cons_Hessian:

$$
\nabla^2 C=(C_1, C_2, C_3, C_4)
=
\left(\begin{bmatrix}
\frac{\partial^2 c_1}{\partial x_1^2} & \frac{\partial^2 c_1}{\partial x_1\partial x_2}\\
\frac{\partial^2 c_1}{\partial x_2\partial x_1} & \frac{\partial^2 c_1}{\partial x_2^2}
\end{bmatrix},
\begin{bmatrix}
\frac{\partial^2 c_2}{\partial x_1^2} & \frac{\partial^2 c_2}{\partial x_1\partial x_2}\\
\frac{\partial^2 c_2}{\partial x_2\partial x_1} & \frac{\partial^2 c_2}{\partial x_2^2}
\end{bmatrix},
\begin{bmatrix}
\frac{\partial^2 c_3}{\partial x_1^2} & \frac{\partial^2 c_3}{\partial x_1\partial x_2}\\
\frac{\partial^2 c_3}{\partial x_2\partial x_1} & \frac{\partial^2 c_3}{\partial x_2^2}
\end{bmatrix},
\begin{bmatrix}
\frac{\partial^2 c_4}{\partial x_1^2} & \frac{\partial^2 c_4}{\partial x_1\partial x_2}\\
\frac{\partial^2 c_4}{\partial x_2\partial x_1} & \frac{\partial^2 c_4}{\partial x_2^2}
\end{bmatrix}\right)
=
\left(\begin{bmatrix}
24x_1^2-48x_1+16 & 0\\
0 & 0
\end{bmatrix},
\begin{bmatrix}
48x_1^2-192x_1+176 & 0\\
0 & 0
\end{bmatrix},
\begin{bmatrix}
0 & 0\\
0 & 0
\end{bmatrix},
\begin{bmatrix}
0 & 0\\
0 & 0
\end{bmatrix}\right)
$$

In [11]:
# TODO: derive and define the functions
def cons_f(x):
    # TODO   
    return np.array([(2 * x[0])**4 - (8 * x[0])**3 + (8 * x[0])**2 - x[1] + 2, 
                     (4 * x[0])**4 - (32 * x[0])**3 + (88 * x[0])**2 - 96 * x[0] - x[1] + 36,
                     -x[0] + 3, 
                     -x[1] + 4
                    ])
    
def cons_Jacobian(x):
    # TODO
    return np.array([[8*x[0]**3 - 24*x[0]**2 + 16*x[0], -1], 
                     [16*x[0]**3 - 96*x[0]**2 + 176*x[0] - 96, -1],
                     [-1, 0], 
                     [0, -1]
                    ])

def cons_Hessian(x,v):
    # TODO
    return v[0]*np.array([
                         [24*x[0]**2 - 48*x[0] + 16, 48*x[0]**2 - 192*x[0] + 176], 
                         [0,0]
                         ])

# TODO: Insert the functions above into a NonlinearConstraint obeject
nonlinear_constraint = NonlinearConstraint(cons_f, 0, np.inf, jac=cons_Jacobian, hess=cons_Hessian)

### Define the bounds 

In [6]:
# TODO: define the bounds
# only a lower bound is needed so the upper bound for both variables is set to np.inf
bounds = Bounds([0,0], [np.inf, np.inf])

### Call the minimize library

In [17]:
# TODO: define initial point
x0 = [0,0]

res = minimize(f, x0, method='trust-constr', constraints=[nonlinear_constraint], bounds=bounds)

/Users/yuxuanlin/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


### Print out the result

In [18]:
print(res.x) 

[0.01847008 0.20855884]


In [19]:
res

 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 206
      cg_stop_cond: 3
            constr: [array([ 1.81005014, 36.45369214,  2.98152992,  3.79144116]), array([0.01847008, 0.20855884])]
       constr_nfev: [218, 0]
       constr_nhev: [14, 0]
       constr_njev: [13, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 0.21616721153259277
               fun: -0.22702892640660663
              grad: array([-1., -1.])
               jac: [array([[  0.28738429,  -1.        ],
       [-92.78191434,  -1.        ],
       [ -1.        ,   0.        ],
       [  0.        ,  -1.        ]]), array([[1., 0.],
       [0., 1.]])]
   lagrangian_grad: array([-0.10014502, -0.43615514])
           message: '`xtol` termination condition is satisfied.'
            method: 'tr_interior_point'
              nfev: 654
              nhev: 0
               nit: 218
             niter: 218
              njev: 218
        optimality:

### Apply COBYLA method

#### calculation process of cons:

transform the constraints to non-negative inequality constraints:

$$\begin{bmatrix}
2x_1^4 - 8x_1^3 + 8x_1^2 - x_2 + 2\\
4x_1^4 - 32x_1^3 + 88x_1^2 - 96x_1 - x_2 + 36\\
-x_1 + 3\\
-x_2 + 4
\end{bmatrix}
\ge
\begin{bmatrix}
0\\
0\\
0\\
0
\end{bmatrix}$$
(COBYLA cannot handle bounds on the variables)


In [22]:
cons = ({'type': 'ineq', 'fun': lambda x: (2 * x[0])**4 - (8 * x[0])**3 + (8 * x[0])**2 - x[1] + 2},
        {'type': 'ineq', 'fun': lambda x: (4 * x[0])**4 - (32 * x[0])**3 + (88 * x[0])**2 - 96 * x[0] - x[1] + 36},
        {'type': 'ineq', 'fun': lambda x: -x[0] + 3},
        {'type': 'ineq', 'fun': lambda x: -x[1] + 4}
       )

In [23]:
# TODO
# Note that COBYLA only supports inequality constraints.
res = minimize(f, x0, method='COBYLA', constraints=cons)
print(res.x)

[-0.12490539  4.        ]


In [24]:
res

     fun: -3.875094608263893
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 25
  status: 1
 success: True
       x: array([-0.12490539,  4.        ])